<a href="https://colab.research.google.com/github/27luiz/dadosSUS/blob/main/Dadosus_AIH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import auth
auth.authenticate_user()


In [14]:
from ftplib import FTP
from google.cloud import storage

def download_ftp_file(ftp_url, local_filename):
    ftp_server = ftp_url.split('/')[2]
    ftp_path = "/".join(ftp_url.split('/')[3:])

    with FTP(ftp_server) as ftp:
        ftp.login()  # Você pode precisar fornecer credenciais de login aqui
        with open(local_filename, 'wb') as f:
            ftp.retrbinary(f'RETR {ftp_path}', f.write)

def upload_to_gcs(local_filename, gcs_bucket_name, gcs_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(gcs_bucket_name)
    blob = bucket.blob(gcs_blob_name)
    blob.upload_from_filename(local_filename)

# URL base do arquivo FTP
ftp_base_url = "ftp://ftp.datasus.gov.br/dissemin/publicos/SIHSUS/200801_/Dados/RD{uf}{ano}{mes}.dbc"

# Localização no Google Cloud Storage para onde o arquivo será carregado
gcs_bucket_name = "tabsus"

# Listas de Ufs, Anos e Meses
ufs = ['MG']
anos = ['23']
meses = ['01', '02', '03']

# Iterar sobre todas as combinações de Ufs, Anos e Meses
for uf in ufs:
    for ano in anos:
        for mes in meses:
            # Construir a URL do FTP substituindo as partes fixas pelas variáveis
            ftp_url = ftp_base_url.format(uf=uf, ano=ano, mes=mes)

            # Construir o nome do arquivo local substituindo as partes fixas pelas variáveis
            local_filename = f"/tmp/RD{uf}{ano}{mes}.dbc"

            # Construir o nome do arquivo no Google Cloud Storage substituindo as partes fixas pelas variáveis
            gcs_blob_name = f"bruto/RD{uf}{ano}{mes}.dbc"

            # Baixe o arquivo FTP
            download_ftp_file(ftp_url, local_filename)

            # Carregue o arquivo baixado para o Google Cloud Storage
            upload_to_gcs(local_filename, gcs_bucket_name, gcs_blob_name)

            print(f"Arquivo enviado com sucesso para o Google Cloud Storage: {gcs_blob_name}")

print("Todos os arquivos foram enviados com sucesso.")

Arquivo enviado com sucesso para o Google Cloud Storage: bruto/RDMG2301.dbc
Arquivo enviado com sucesso para o Google Cloud Storage: bruto/RDMG2302.dbc
Arquivo enviado com sucesso para o Google Cloud Storage: bruto/RDMG2303.dbc
Todos os arquivos foram enviados com sucesso.
